In [10]:
!pip install git+https://github.com/openai/CLIP.git
!pip install open_clip_torch
!pip install sentence_transformers


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-wgufvkqr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-wgufvkqr
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [11]:
!pip install opencv-python

In [12]:
!pip install shap

In [13]:
import numpy as np
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
#from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import KFold
#from classification_models.tfkeras import Classifiers
import torch
import open_clip
import cv2
from sentence_transformers import util
from PIL import Image

In [14]:
orig_prompt = 'a lamp with flowers'

sub prompts:

    
    a lamp
    with flowers
    lamp with
    a lamp flowers
    lamp with
    a lamp with
    lamp flowers
    lamp with flowers
    with lamp a flowers
    

## Calculate similarities of each image with the original image

In [15]:


# image processing model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model.to(device)
def imageEncoder(img):
    img1 = Image.fromarray(img).convert('RGB')
    img1 = preprocess(img1).unsqueeze(0).to(device)
    img1 = model.encode_image(img1)
    return img1
def generateScore(image1, image2):
    test_img = cv2.imread(image1, cv2.IMREAD_UNCHANGED)
    data_img = cv2.imread(image2, cv2.IMREAD_UNCHANGED)
    img1 = imageEncoder(test_img)
    img2 = imageEncoder(data_img)
    cos_scores = util.pytorch_cos_sim(img1, img2)
    score = round(float(cos_scores[0][0])*100, 2)
    return score

In [16]:
orig_image = 'a_lamp_with_flowers.jpg'
sub1 = 'a_lamp.jpg'
sub2 = 'a_lamp_flowers.jpg'
sub3 = 'with_flowers.jpg'

In [17]:
print(f"Similarity Score of the image from 'a lamp': ", round(generateScore(orig_image, sub1), 2))

Similarity Score of the image from 'a lamp':  62.44


In [18]:
print(f"Similarity Score of the image from 'a lamp flowers': ", round(generateScore(orig_image, sub2), 2))

Similarity Score of the image from 'a lamp flowers':  69.55


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [19]:
print(f"Similarity Score of the image from 'with flowers': ", round(generateScore(orig_image, sub3), 2))

Similarity Score of the image from 'with flowers':  47.72



## Calculate shap values of prompt on the basis of the similarity metrics

## Using the Gradient Explainer:
Expected gradients combines ideas from Integrated Gradients, SHAP, and SmoothGrad into a single expected value equation. This allows an entire dataset to be used as the background distribution (as opposed to a single reference value) and allows local smoothing. If we approximate the model with a linear function between each background data sample and the current input to be explained, and we assume the input features are independent then expected gradients will compute approximate SHAP values. In the example below we have explained how the 7th intermediate layer of the VGG16 ImageNet model impacts the output probabilities.

## Kernel Expainer

An implementation of Kernel SHAP, a model agnostic method to estimate SHAP values for any model. Because it makes no assumptions about the model type, KernelExplainer is slower than the other model type specific algorithms.



###LIME

In [21]:
prompt = 'a lamp with flowers'
# Have
img = 'a_lamp_with_flowers.jpg'

a_prompt = []
lamp_prompt = []
with_prompt = []
flowers_prompt = []

# Have
parse1 = 'a_lamp_with.jpg'
score = generateScore(img, parse1)
a_prompt.append(score)
lamp_prompt.append(score)
with_prompt.append(score)

# Have
parse2 = 'a_lamp_flowers.jpg'
score = generateScore(img, parse2)
a_prompt.append(score)
lamp_prompt.append(score)
flowers_prompt.append(score)

# Have
parse3 = 'a_with_flowers.jpg'
score = generateScore(img, parse3)
a_prompt.append(score)
with_prompt.append(score)
flowers_prompt.append(score)

# Have
parse4 = 'a_lamp.jpg'
score = generateScore(img, parse4)
a_prompt.append(score)
lamp_prompt.append(score)

# Have
parse5 = 'a_with.jpg'
score = generateScore(img, parse5)
a_prompt.append(score)
with_prompt.append(score)

# Have
parse6 = 'a_flowers.jpg'
score = generateScore(img, parse6)
a_prompt.append(score)
flowers_prompt.append(score)

# Have (duplicated this one - not legit)
parse7 = 'a.jpg'
score = generateScore(img, parse7)
a_prompt.append(score)

# Have
parse8 = 'lamp_with_flowers.jpg'
score = generateScore(img, parse8)
lamp_prompt.append(score)
with_prompt.append(score)
flowers_prompt.append(score)

# Have
parse9 = 'lamp_with.jpg'
score = generateScore(img, parse9)
lamp_prompt.append(score)
with_prompt.append(score)

# Have
parse10 = 'lamp_flowers.jpg'
score = generateScore(img, parse10)
lamp_prompt.append(score)
flowers_prompt.append(score)

# Have
parse11 = 'lamp.jpg'
score = generateScore(img, parse11)
lamp_prompt.append(score)

# Have
parse12 = 'with_flowers.jpg'
score = generateScore(img, parse12)
with_prompt.append(score)
flowers_prompt.append(score)

# Have (duplicated this one - not legit)
parse13 = 'with.jpg'
score = generateScore(img, parse13)
with_prompt.append(score)

parse14 = 'flowers.jpg'
score = generateScore(img, parse14)
flowers_prompt.append(score)



In [ ]:
a_prompt = sum(a_prompt)
lamp_prompt = sum(lamp_prompt)
with_prompt = sum(with_prompt)
flowers_prompt = sum(flowers_prompt)

In [ ]:
print("A: ", a_prompt)
print("Lamp: ", lamp_prompt)
print("With: ", with_prompt)
print("Flowers", flowers_prompt)

In [ ]:
!pip install lime

In [ ]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christian']

In [ ]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

In [ ]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train_vectors, newsgroups_train.target)

In [ ]:
pred = rf.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred, average='binary')

LIME Explanations

In [ ]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)
print(c.predict_proba([newsgroups_test.data[0]]))

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
idx = 83
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(christian) =', c.predict_proba([newsgroups_test.data[idx]])[0,1])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

In [ ]:
exp.as_list()

In [ ]:
print('Original prediction:', rf.predict_proba(test_vectors[idx])[0,1])
tmp = test_vectors[idx].copy()
tmp[0,vectorizer.vocabulary_['Posting']] = 0
tmp[0,vectorizer.vocabulary_['Host']] = 0
print('Prediction removing some features:', rf.predict_proba(tmp)[0,1])
print('Difference:', rf.predict_proba(tmp)[0,1] - rf.predict_proba(test_vectors[idx])[0,1])

In [ ]:
print('Original prediction:', rf.predict_proba(test_vectors[idx])[0,1])
tmp = test_vectors[idx].copy()
tmp[0,vectorizer.vocabulary_['Posting']] = 0
tmp[0,vectorizer.vocabulary_['Host']] = 0
print('Prediction removing some features:', rf.predict_proba(tmp)[0,1])
print('Difference:', rf.predict_proba(tmp)[0,1] - rf.predict_proba(test_vectors[idx])[0,1])

In [ ]:
%matplotlib inline
fig = exp.as_pyplot_figure()

In [ ]:
exp.show_in_notebook(text=False)

In [ ]:
exp.save_to_file('/tmp/oi.html')


In [ ]:
exp.show_in_notebook(text=True)
